In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
!pip install nltk



  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------- ----- 1.3/1.5 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 5.0 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [4]:
data = pd.read_csv("sms_spam.csv")

In [5]:
data.head()

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data.describe()

,type,text
count,5574,5574
unique,2,5160
top,ham,"Sorry, I'll call later"
freq,4827,30


In [7]:
data["text"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5569    This is the 2nd time we have tried 2 contact u...
5570                 Will ü b going to esplanade fr home?
5571    Pity, * was in mood for that. So...any other s...
5572    The guy did some bitching but I acted like i'd...
5573                           Rofl. Its true to its name
Name: text, Length: 5574, dtype: object

In [14]:
import pandas as pd
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

# Run this once if not downloaded
# nltk.download('stopwords')

ps = PorterStemmer()

def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'http\S+', '', text)  # remove links
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = text.strip()

    

    return text




In [16]:
for i in range(len(data["text"])):
    data["text"][i] = clean_text(data["text"][i])

In [17]:
data.head()

,type,text
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample text data


# Initialize the vectorizer
vectorizer = CountVectorizer()

# Fit and transform the data
X = vectorizer.fit_transform(data["text"])

# Convert to array or DataFrame for readability
print(vectorizer.get_feature_names_out())
print(X.toarray())

['aa' 'aah' 'aaniye' ... 'zyada' 'üll' '〨ud']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [20]:
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 71016 stored elements and shape (5574, 8592)>

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [23]:
df = pd.read_csv("sms_spam.csv")
df.columns = ["label","message"]
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
df["label_num"] = df["label"].map({"ham":0,"spam":1})

In [27]:
df.head()

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [29]:
X_train,X_test,y_train,y_test = train_test_split(df["message"],df["label_num"],test_size=0.2,random_state=42)

In [30]:
X_train.head()

1350    FREE2DAY sexy St George's Day pic of Jordan!Tx...
5544             Armand says get your ass over to epsilon
1168              Lol now I'm after that hot air balloon!
5551    You know, wot people wear. T shirts, jumpers, ...
5320    Good morning, my Love ... I go to sleep now an...
Name: message, dtype: object

In [31]:
vectorizer = TfidfVectorizer(stop_words= "english")

In [32]:
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.fit_transform(X_test)

In [34]:
X_train_vec

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 34695 stored elements and shape (4459, 7463)>

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

# Step 1: Fit on training data
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

# Step 2: Transform test data using the same vectorizer
X_test_vec = vectorizer.transform(X_test)



In [38]:
model = MultinomialNB()
model.fit(X_train_vec,y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [39]:
# Step 3: Train and predict
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)

In [40]:
print("Accuracy: ",accuracy_score(y_test,y_pred))
print("\nClassification Report: \n",classification_report(y_test,y_pred))
print("\nConfusion Matrix: \n",confusion_matrix(y_test,y_pred))

Accuracy:  0.9856502242152466

Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       954
           1       0.97      0.93      0.95       161

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115


Confusion Matrix: 
 [[949   5]
 [ 11 150]]


In [41]:
def check_spam(text):
    text_vec = vectorizer.transform([text])
    prediction = model.predict(text_vec)[0]
    return "Spam" if prediction == 1 else "Ham"

print(check_spam("Congratulations! You won a lottery"))
print(check_spam("Hey, are we meeting today?"))

Spam
Ham
